<span style="font-size: 14pt">MIPT, Advanced ML, Autumn 2017</span>

<span style="font-size: 16pt"> HW #9: Recommendation System

<span style="color:blue; font-size: 12pt">Andrey Saitgalin </span>
<span style="color:blue; font-size: 12pt; font-family: 'Verdana'">andrey.saitgalin@gmail.com</span>, 

<span style="color:blue; font-size: 12pt">Alexey Dral </span>,
<span style="color:blue; font-size: 12pt; font-family: 'Verdana'">aadral@gmail.com</span>

<h1 align="center">Organization Info</h1> 

**Дополнительный материал для выполнения дз**:
<a href= "https://github.com/ml-mipt/ml-mipt-part2/tree/master/2017/lectures">Лекции по курсу</a>

**Оформление дз**: 
- Присылайте выполненное задание на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2017_Aut_fall <номер_группы> <фамилия>``, к примеру -- ``ML2017_Aut_fall 401 ivanov``
- Выполненное дз сохраните в файл ``<фамилия>_<группа>_task<номер>.ipnb``, к примеру -- ``ivanov_491_task9.ipnb``

**Вопросы**:
- Присылайте вопросы на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2017_Aut_fall Question <Содержание вопроса>``

** Дедлайн**:
<span style="color:red; font-size: 12pt">16.12.2017 23:59 </span>

--------
- **PS1**: Мы используем автоматические фильтры, и просто не найдем ваше дз, если вы не аккуратно его подпишите.
- **PS2**: Напоминаем, что дедлайны жесткие, письма пришедшие после автоматически удаляются =( чтобы соблазна не было.

<h1 align="center">Check Questions (0%)</h1> 
Вопросы отсутствуют

<h1 align="center"> Practical tasks (100%)</h1>


# Построение content-based рекомендательной системы
## Part 1
Для каждого фильма получите описание в текстовом виде с IMDB или (и) The movie database (у них есть открытое API). https://www.themoviedb.org/?language=ru. Вывести описание фильма под id 778. Также можно использовать рецензии.

In [52]:
import numpy as np
import pandas as pd
from collections import Counter, defaultdict
from tqdm import tqdm
from scipy.spatial.distance import cosine
from scipy.sparse import csr_matrix

In [2]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
links = pd.read_csv('links.csv')
tmdb_data = pd.read_csv(open('tmdb_data.csv','rU'), encoding='utf-8', engine='c')
links = links.dropna(axis=0, how='any')
links.tmdbId = links.tmdbId.astype(np.int64)
tmdb_data = tmdb_data.dropna(axis=0, how='any')
tmdb_data = tmdb_data[tmdb_data.id.str.match('\d+')]
tmdb_data.id = tmdb_data.id.astype(np.int64)
tmdb_data = tmdb_data.groupby('id').last().reset_index()
udf = ratings[['userId', 'movieId', 'rating']]
mdf = movies[['movieId', 'genres']].merge(links[['movieId', 'tmdbId']], on='movieId')
mdf = mdf.merge(tmdb_data[['id', 'overview']], left_on='tmdbId', right_on='id')
mdf.drop(['tmdbId', 'id'], axis=1, inplace=True)
mdf.set_index('movieId', inplace=True)
mdf['num'] = np.arange(len(mdf))
mdf = mdf[mdf.num < 1000]
udf = udf[udf.movieId.isin(mdf.index)]

/usr/lib64/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 'U' mode is deprecated
  after removing the cwd from sys.path.


In [6]:
mdf.loc[778].overview

'Renton, deeply immersed in the Edinburgh drug scene, tries to clean up and get out, despite the allure of the drugs and influence of friends.'

## Part 2 
Извлеките текстовые признаки у фильмов: tf idf, биграммы (может какие-то еще). Биграммы - статистика по биграммам, сколько какая биграмма встречается. Формулы для TF-IDF смотреть тут: https://en.wikipedia.org/wiki/Tf%E2%80%93idf 
    1. Используйте TF с double 0.5 normalization
    2. Используйте стандартный IDF: 
$$idf(t, D) = \frac{log N}{(1 + |d in D : t in d|)}$$

In [35]:
class MyTfidfVectorizer:
    def __init__(self, docs):
        unigram_voc = {}
        bigram_voc = {}
        
        for doc in tqdm(docs):
            words = doc.split()
            for i, word in enumerate(words):
                if i + 1 != len(words):
                    pair = (word, words[i + 1])
                    if pair not in bigram_voc:
                        bigram_voc[pair] = len(bigram_voc)
                
                if word not in unigram_voc:
                    unigram_voc[word] = len(unigram_voc)
        
        self.tf = csr_matrix((len(docs), len(unigram_voc)))
        self.idf = np.zeros(len(unigram_voc))
        self.bigram = csr_matrix((len(docs), len(bigram_voc)))
        
        for i, doc in tqdm(enumerate(docs)):
            words = doc.split()
            
            cntr = Counter(words)
            for word, count in cntr.items():
                self.tf[i, unigram_voc[word]] = count / len(words)
                self.idf[unigram_voc[word]] += 1
                
            for j in range(len(words) - 1):
                self.bigram[i, bigram_voc[(words[j], words[j + 1])]] += 1

        self.idf = np.log(len(docs) / self.idf)
    
    def __getitem__(self, i):
        row = self.tf.getrow(i).toarray()[0]
        result = 0.5 * (1 + row / row.max())
        return np.append(result, self.bigram.getrow(i).toarray()[0])

In [39]:
tfidf = MyTfidfVectorizer(mdf.overview.tolist())

100%|██████████| 1000/1000 [00:00<00:00, 12325.31it/s]
0it [00:00, ?it/s]/home/mikhail/.local/lib64/python3.6/site-packages/scipy/sparse/compressed.py:742: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)
1000it [01:27, 11.45it/s]


## Part 3 
Выведите подсчитанные признаки из пункта 2 для фильма с id 778.

In [40]:
tfidf[778]

array([ 0.5,  0.5,  0.5, ...,  0. ,  0. ,  0. ])

## Part 4
Для каждого пользователя найдите топ жанров, которые нравятся пользователю (можно делать как на семинаре top 5, top 3 и top 2) и порекомендуйте таким пользователя фильмы близкие по текстовому описанию (см. пункт 2). Для оценки результатов пункта 3 используйте метрику HitRate и метрики ранжирования (N)DCG. Формулу для подсчета HitRate можно взять на семинаре,  (N)DCG: https://en.wikipedia.org/wiki/Discounted_cumulative_gain

